In [136]:
import numpy as np
import sys
sys.path.append("../src")
from bert_embedding import *

# Data

## 1. SEC filings

In [4]:
test_link ="https://github.com/juand-r/entity-recognition-datasets/tree/master/data/SEC-filings/CONLL-format/data/test/FIN3"
train_link = "https://github.com/juand-r/entity-recognition-datasets/tree/master/data/SEC-filings/CONLL-format/data/train/FIN5.txt"

In [19]:
# !wget https://raw.githubusercontent.com/juand-r/entity-recognition-datasets/master/data/SEC-filings/CONLL-format/data/test/FIN3.txt -P ../data/sec_ner

In [20]:
# !wget https://raw.githubusercontent.com/juand-r/entity-recognition-datasets/master/data/SEC-filings/CONLL-format/data/train/FIN5.txt -P ../data/sec_ner

## 2. BTC

In [21]:
# !wget https://raw.githubusercontent.com/juand-r/entity-recognition-datasets/master/data/BTC/CONLL-format/data/h.conll -P ../data/ner_btc

## 3. Wikigold

In [23]:
# !wget https://raw.githubusercontent.com/juand-r/entity-recognition-datasets/master/data/wikigold/CONLL-format/data/wikigold.conll.txt -P ../data/ner_wikigold

# Data Clean

In [243]:
# load data
def load_ner_data(path, separator = " "):
    with open(path) as f:
        text = f.read().split("\n\n")

    output = []
    for line in text:
        feature_label = []
        line = line.split("\n")
        for entry in line:
            feature_label.append(tuple(entry.split(separator)))
        output.append(feature_label)
    return output

# get words and tags
def unique_words_tags(data):
    unique_words = []
    unique_tags = []
    for sent in data:
        unique_words.extend(list(set(np.array(sent)[:,0])))
        unique_tags.extend(list(set(np.array(sent)[:,-1])))
        
    return set(unique_words), set(unique_tags)

# get words and tags distributions
def distributions_words_tags(data):
    unique_words = {}
    unique_tags = {}
    for i in range(len(data)-1):
        sent = data[i]
        for t in sent:
            word = t[0]
            tag = t[-1]
            
            if word in unique_words:
                unique_words[word] += 1
            else:
                unique_words[word] = 1
                
            if tag in unique_tags:
                unique_tags[tag] += 1
            else:
                unique_tags[tag] = 1
                
    return unique_words, unique_tags

In [244]:
a = []
a.extend(list(set(["1","2","3"])))
a

['3', '1', '2']

In [142]:
sec_path = "../data/ner_sec/FIN5.txt"
sec = load_ner_data(sec_path)

In [143]:
print(len(sec))
# sec[1]

1170


In [144]:
btc = []
for data in ["a", "b", "e", "f", "g", "h"]:
    btc.extend(load_ner_data("../data/ner_btc/" + data + ".conll", "\t"))
print(len(btc))
# btc[1]

9345


In [145]:
wiki = load_ner_data("../data/ner_wikigold/" + "wikigold" + ".conll.txt", " ")
print(len(wiki))
# wiki[1]

1842


In [236]:
wiki[1][0][-1]

'O'

In [245]:
for data in [wiki, sec, btc]:
    print(distributions_words_tags(data)[1])
# only wiki and sec works here

{'I-MISC': 1392, 'O': 32721, 'I-ORG': 1958, 'I-PER': 1634, 'I-LOC': 1447}
{'O': 39485, 'I-ORG': 384, 'I-LOC': 356, 'I-PER': 783, 'I-MISC': 7}
{'O': 131814, 'B-LOC': 2822, 'B-PER': 7928, 'B-ORG': 4135, 'I-ORG': 1176, 'I-PER': 1554, 'I-LOC': 958, '': 5}


In [246]:
for sent in wiki:
    for t in sent:
        if t[-1] =="":
            print(sent)

[('',)]


In [365]:
def sent2labels(sent):
    return [label for token, label in sent]

In [367]:
def sent2labels1(sent):
    return [t[-1] for t in sent]

In [369]:
sent2labels(wiki[0])

['I-MISC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'I-MISC',
 'O',
 'O',
 'O',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'O']

In [370]:
sent2labels1(wiki[0])

['I-MISC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'I-MISC',
 'O',
 'O',
 'O',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'O']

In [419]:
def word2features(sent, i):
    """
    The function generates all features
    for the word at position i in the
    sentence.
    """
    word = sent[i][0]
    f = tokenize_encode_bert_sentences_sample(tokenizer_d, model_d, word)[0]
    features = {}
    for j in range(len(f)):
        features[str(j)] = f[j]
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [377]:
labels = [sent2labels1(s) for s in wiki]

In [374]:
features = [sent2features(s) for s in wiki]

In [392]:
# features[0:2]

In [438]:
haha = tokenize_encode_bert_sentences_sample(tokenizer_d, model_d, "haha")

In [437]:
a = sec[-2:]
a

[[('/', 'NN', '-', 'O'),
  ('s', 'NNS', '-', 'O'),
  ('/', ':', '-', 'O'),
  ('Bing', 'VBG', '-', 'I-PER'),
  ('Yu', 'NNP', '-', 'I-PER')],
 [('',)]]

In [442]:
# haha[0]
# haha_f = {}
# for j in range(len(haha[0])):
#     haha_f[str(j)] = haha[0][j]
# haha_f

# Model

https://towardsdatascience.com/named-entity-recognition-ner-meeting-industrys-requirement-by-applying-state-of-the-art-deep-698d2b3b4ede
https://www.depends-on-the-definition.com/sequence-tagging-lstm-crf/

In [135]:
tokenizer_d = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model_d = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [317]:
words_wiki, tags = unique_words_tags(wiki)
words_sec, tags = unique_words_tags(sec)
print(len(words_wiki), len(words_sec), len(words_wiki | words_sec))

8506 3513 10694


In [284]:
sent_lens = [len(x) for x in wiki]
print(max([len(x) for x in wiki]), max([len(x) for x in sec]))

144 413


In [460]:
max_len = 200
words_wiki.remove("")
words_sec.remove("")
word2idx = {w: i + 1 for i, w in enumerate(words_wiki | words_sec)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [291]:
from keras.utils import to_categorical
y_wiki = [to_categorical(i, num_classes=len(tags)) for i in y]
y_wiki[0][0]

y_sec = [to_categorical(i, num_classes=len(tags)) for i in y]
y_sec[0][0]

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)

In [453]:
from sklearn.model_selection import train_test_split
train_wiki, test_wiki = train_test_split(wiki)

In [449]:
encoded_ner_corpus = tokenize_encode_bert_sentences(tokenizer_d, model_d, words_list, "../data/all_bert/encoded_ner_corpus")

In [470]:
tokenize_encode_bert_sentences_sample(tokenizer_d, model_d, ["Randall"])[0][:10]

array([-0.28524143, -0.04883502, -0.08547825, -0.10694244,  0.0716219 ,
       -0.06495161,  0.08082949,  0.10530194, -0.15842576, -0.17903239],
      dtype=float32)

In [471]:
encoded_ner_corpus[0][:10]

array([-0.28524143, -0.04883502, -0.08547825, -0.10694244,  0.0716219 ,
       -0.06495161,  0.08082949,  0.10530194, -0.15842576, -0.17903239])

In [477]:
ner_bert = np.load("../data/all_bert/encoded_ner_corpus.npy")

In [489]:
ner_bert[word2idx["Randall"]-1]

array([-2.85241425e-01, -4.88350242e-02, -8.54782462e-02, -1.06942438e-01,
        7.16219023e-02, -6.49516061e-02,  8.08294863e-02,  1.05301939e-01,
       -1.58425763e-01, -1.79032385e-01,  1.42368212e-01, -1.00047708e-01,
        3.94101627e-02,  2.55815744e-01,  9.67776552e-02,  2.38357902e-01,
       -1.02056831e-01,  1.29859746e-01,  1.72019944e-01, -3.02408785e-01,
       -6.49108067e-02, -3.54369313e-01, -6.31042942e-02, -1.61043271e-01,
       -6.03366308e-02,  1.21796429e-01,  7.48223066e-03, -1.32442921e-01,
        1.24639072e-01,  1.23222195e-01,  8.35943893e-02,  9.84147266e-02,
        5.34838438e-02,  5.47583029e-02,  7.18373805e-02,  6.06911294e-02,
        4.88229580e-02, -6.60748258e-02,  2.28404880e-01,  5.22120148e-02,
        3.14897597e-02, -8.63815546e-02,  2.17607155e-01, -6.94252774e-02,
       -9.89933684e-02,  3.35037932e-02, -1.87032449e+00, -8.31993669e-02,
       -1.43446892e-01, -8.84553492e-02,  5.20219505e-02,  9.51768905e-02,
        1.42228246e-01,  